<a href="https://colab.research.google.com/github/edcalderin/DeepLearning_SaturdaysAI/blob/master/3_RecurrentNeuralNets/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word embeddings

En la parte teórica hemos visto que para crear *neural networks* para el lenguaje, primero tenemos que transformar el texto en una matriz de vectores. El tipo de vectores que capturan la semántica de las palabras se llaman **word embeddings** y son los que se utilizan en la mayoría de aplicaciones modernas de NLP. Vamos a ver cómo utilizamos embeddings en PyTorch.  

En PyTorch se pueden utilizan los *word embeddings* en inglés más típicos (word2vec, GloVe, FastText...) directamente. Nosotros vamos a trabajar con *embeddings* españoles. Recomiendo bajarse estos vectores https://www.kaggle.com/rtatman/pretrained-word-vectors-for-spanish creados por Cristian Cardellino (atención, es un fichero grande de aprox. 3GB)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading

Loading word embeddings en inglés es muy fácil. PyTorch da acceso a los embeddings más típicos a través de la library **torchtext**. 

In [2]:
# load english vectors
from torchtext import vocab

glove = vocab.GloVe(name='6B', dim=100)

print("Hay {} palabras en el vocabulario".format(len(glove.itos)))

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 398979/400000 [00:17<00:00, 23367.94it/s]

Hay 400000 palabras en el vocabulario


Para acceder a los vectores españoles, tenemos que indicar dónde está el fichero.

In [74]:
!unzip -u "drive/MyDrive/SaturdaysAI/words_espanioles.zip" -d "drive/MyDrive/SaturdaysAI/"

Archive:  drive/MyDrive/SaturdaysAI/words_espanioles.zip
  inflating: drive/MyDrive/SaturdaysAI/SBW-vectors-300-min5.txt  


In [5]:
# load spanish vectors

es_vectors = vocab.Vectors('SBW-vectors-300-min5.txt', cache='drive/MyDrive/SaturdaysAI')

print("Hay {} palabras en el vocabulario".format(len(es_vectors.itos)))


  0%|          | 0/1000653 [00:00<?, ?it/s]Skipping token b'1000653' with 1-dimensional vector [b'300']; likely a header

100%|█████████▉| 1000530/1000653 [02:05<00:00, 8137.37it/s]

Hay 1000653 palabras en el vocabulario


In [75]:
!rm "drive/MyDrive/SaturdaysAI/SBW-vectors-300-min5.txt"

In [73]:
# ver dimensiones

es_vectors.vectors.shape

torch.Size([1000653, 300])

# Examinar los embeddings

Podemos examinar los embeddings individualmente y ver qué vector está asociado con qué palabra del vocabulario. 

In [40]:
# encontrar el índice de una palabra

es_vectors.stoi['perro']

5880

In [32]:
# examinar vector

es_vectors.vectors[5880]

tensor([ 0.0294, -0.0767, -0.0595,  0.0731,  0.0278, -0.0068,  0.0362, -0.0951,
         0.0946, -0.0277,  0.0045, -0.0111, -0.0389, -0.0595, -0.0174, -0.0377,
        -0.0026,  0.0027, -0.0290,  0.0963,  0.0658, -0.0475,  0.0155,  0.0125,
        -0.0834, -0.0045,  0.0529,  0.0823, -0.0269, -0.0868,  0.0516,  0.0223,
         0.0586,  0.0049,  0.0703, -0.0123,  0.0627,  0.0255, -0.0643,  0.0227,
        -0.0260, -0.0200,  0.0602,  0.0752, -0.0453,  0.0394,  0.0571, -0.0208,
         0.0015, -0.0030,  0.0156,  0.0337,  0.0745,  0.0540,  0.0113, -0.0363,
        -0.0499,  0.0248, -0.0133,  0.0023, -0.0112,  0.0279,  0.0392, -0.0680,
        -0.0473,  0.0227, -0.0683,  0.1432, -0.1118, -0.0679, -0.0341,  0.0041,
        -0.0384, -0.0034, -0.0983,  0.0948, -0.0097,  0.0620,  0.0718, -0.0653,
         0.0605, -0.0886, -0.0101,  0.0331,  0.0281,  0.0428, -0.0267,  0.0358,
         0.0687,  0.0099, -0.0191,  0.0494, -0.0217, -0.0545, -0.0178, -0.0458,
         0.0210, -0.0035, -0.0446, -0.03

In [16]:
# out-of-vocabulary words

es_vectors.stoi['perro']

5880

In [47]:
# funcion que extrae el word embedding para una palabra

def get_vector(embeddings, word):
    assert word in embeddings.stoi, f'*{word}* no se encuentra en el vocabulario!'
    return embeddings.vectors[embeddings.stoi[word]]

In [48]:
# examinar vector

get_vector(es_vectors, 'perro').shape

torch.Size([300])

# Contextos similares

Para encontrar palabras similares a una palabra en concreto, primero tenemos que encontrar el vector de esta palabra y luego calcular la distancia entre este vector y los vectores del resto de las palabras. Luego los ordenamos de más cerca a más lejano. 

In [67]:
# Función para encontrar palabras más similares

import torch
def closest_words(embeddings, vector, n = 10):
    
    distances = [(word, torch.dist(vector, get_vector(embeddings, word)).item())
                 for word in embeddings.itos]
    
    return sorted(distances, key = lambda w: w[1])[:n]

In [68]:
# Buscar vectores más cercanos

word_vector = get_vector(es_vectors, 'perro')

closest_words(es_vectors, word_vector)

[('perro', 0.0),
 ('perros', 0.7023846507072449),
 ('cachorro', 0.7027554512023926),
 ('gato', 0.7147189378738403),
 ('schnauzer', 0.7251959443092346),
 ('mastín', 0.7283346056938171),
 ('caniche', 0.7307871580123901),
 ('teckel', 0.7311853170394897),
 ('pinscher', 0.7341269850730896),
 ('collie', 0.7447739243507385)]

# Analogía

Otra propiedad de los *word embeddings* es que podemos hacer operaciones como si fueran vectores normales, con resultados interesantes.

In [69]:
def analogy(embeddings, word1, word2, word3, n=5):
    
    #obtener vectores para cada palaba
    word1_vector = get_vector(embeddings, word1)
    word2_vector = get_vector(embeddings, word2)
    word3_vector = get_vector(embeddings, word3)
    
    #calcularel vector análogo
    analogy_vector = word2_vector - word1_vector + word3_vector
    
    #encontrar palabras más cercanas
    candidate_words = closest_words(embeddings, analogy_vector, n+3)
    
    #filtrar palabras que ya se encuentran en la analogía
    candidate_words = [(word, dist) for (word, dist) in candidate_words 
                       if word not in [word1, word2, word3]][:n]
    
    print(f'{word1} es a {word2} como {word3} es a...')
    
    return candidate_words

In [70]:
def print_tuples(tuples):
    for w, d in tuples:
        print(f'({d:02.04f}) {w}')

In [71]:
# buscar analogía

print_tuples(analogy(es_vectors, "rey", "hombre", "reina"))
print_tuples(analogy(es_vectors, 'perro', 'cachorro', 'gato'))

rey es a hombre como reina es a...
(0.8070) mujer
(0.9641) joven
(0.9784) fémina
(0.9864) jovencita
(1.0313) muchacha
perro es a cachorro como gato es a...
(0.9114) gatito
(0.9396) oso
(0.9726) cachorros
(0.9743) suricato
(0.9747) cachorrito
